In [1]:
import shortuuid
import names
import mariadb
import sys
import pandas as pd

In [2]:
# Connect to MariaDB Platform
try:
    conn = mariadb.connect(
        user="root",
        password="AXOYu0mzs8vWyMGTBPLl",
        host='containers-us-west-16.railway.app',
        port=5604,
        database="railway"

    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

# Get Cursor
cur = conn.cursor()

In [3]:
cur.execute("SHOW tables;")
rows = cur.fetchall()
df_tables = pd.DataFrame(data=rows, columns=['TABLES'])
df_tables

,TABLES
0,EXPENSE
1,HAS_GROUP
2,USER
3,USER_FRIEND
4,USER_HAS_GROUP_EXPENSE


# INSERT USER

In [4]:
# INSERT USER
for i in range(25):
    id = shortuuid.uuid()
    name = names.get_full_name()
    cur.execute(
    "INSERT INTO USER (user_id, user_name) VALUES (?, ?);", 
    (id, name))

In [5]:
cur.execute("SELECT * FROM USER;")
rows = cur.fetchall()
df_user = pd.DataFrame(data=rows, columns=['user_id', 'user_name'])
df_user

,user_id,user_name
0,33PYkZmvXrhDTaSJmWoZUE,Ursula Abney
1,4bLSJCzTxZJSLkR8ejYmfq,Robert Potts
2,a2bbwktZh2V63UQkuqy4rL,Joseph Williams
3,a2yTZVhHDsRvJFSVjPHtTk,Robert Kleist
4,azS6juetzqoB8R9fiotLEW,Diane Henry
5,BArzWmDEn5gQjSgNS9ZKs8,Eleanor Franklin
6,CmAiFWonnhqHG3VB7NMu8z,David Fox
7,Cq6Md3DbLx3Rs5zmShrYjP,Mary Shulman
8,dAQwzYmQ8a2ewtYyCacLas,Doris Duffy
9,fmNamvhQW7gV6CzrNL9oB3,Kenny Richards


# INSERT USER_FRIEND 

In [7]:
# getting the ids
cur.execute('SELECT user_id FROM USER;')
rows = cur.fetchall()
df_user_id = pd.DataFrame(data=rows, columns=['user_id'])

In [8]:
import random
list_user_id = df_user_id.user_id
for user_id in list_user_id:
    l = list_user_id.tolist()
    l.remove(user_id)
    p = random.sample(l, 5)
    # print(p)
    for u in p: 
        cur.execute(
        "INSERT INTO USER_FRIEND (user_id, friend) VALUES (?, ?);", 
        (user_id, u))

In [13]:
cur.execute("SELECT * FROM USER_FRIEND;")
rows = cur.fetchall()
df_user = pd.DataFrame(data=rows, columns=['user_id', 'friend'])
df_user

,user_id,friend
0,33PYkZmvXrhDTaSJmWoZUE,dAQwzYmQ8a2ewtYyCacLas
1,33PYkZmvXrhDTaSJmWoZUE,JpSwyWniiEtCxZnbTrNtNi
2,33PYkZmvXrhDTaSJmWoZUE,KbGpvPrUZBRGHXTPXtFma2
3,33PYkZmvXrhDTaSJmWoZUE,oWyQyMitT5F3eh4sBSpp4h
4,33PYkZmvXrhDTaSJmWoZUE,Rs7KBAJa9iKEDRPaUWWLgU
...,...,...
120,UokgG3NcswXj9cB9eWteoS,a2bbwktZh2V63UQkuqy4rL
121,UokgG3NcswXj9cB9eWteoS,GJzvRUTCnS8t2SG6WU5GQb
122,UokgG3NcswXj9cB9eWteoS,hrFLmvibWjvuMXKS2d2LrD
123,UokgG3NcswXj9cB9eWteoS,M29UWE2Z3uvXxbZK9tZYFm


# INSERT HAS_GROUP, USER_HAS_GROUP_EXPENSE, EXPENSE 

In [9]:
# getting the ids
cur.execute('SELECT user_id FROM USER;')
rows = cur.fetchall()
df_user_id = pd.DataFrame(data=rows, columns=['user_id']).user_id.tolist()

In [10]:
from faker import Faker
import datetime
fake = Faker()

exp_date = fake.date_time_between(start_date='-1y', end_date='now').strftime('%Y-%m-%d')
exp_date

'2023-02-16'

In [15]:
from mariadb.constants import *

groupno=1
for user_id in df_user_id:
    cur.execute("SELECT friend FROM USER_FRIEND where user_id= ? ;",(user_id,)) # getting the friends of the user
    rows = cur.fetchall()
    df_user_id = pd.DataFrame(data=rows, columns=['user_id']).user_id.tolist() # getting the list

    # random generating of group expsense
    x=random.randrange(1,10)
    for i in range(x):
        group_id = shortuuid.uuid()     
        
        samples  = random.randrange(1,5) # for getting the number of debtor 
        p = random.sample(df_user_id, samples) # getting the random debtors
          
        amount = random.randrange(100,1000)
        group_name =  'Group {}'.format(groupno) # if (randombool) return null else group no.
        groupno+=1
        
        # INSERT HAS_GROUP
        cur.execute("INSERT INTO HAS_GROUP VALUES (? , ? );", (group_id, group_name)) 

        # INSERT USER_HAS_GROUP_EXPENSE
        exp_date = fake.date_time_between(start_date='-1y', end_date='now').strftime('%Y-%m-%d')
        
        expense_id = shortuuid.uuid() # expense id of the creditor
        cur.execute("INSERT INTO USER_HAS_GROUP_EXPENSE VALUES (? , ? , ?);", (user_id, group_id, expense_id)) 
        cur.execute("INSERT INTO EXPENSE VALUES (?, ?, ?, ?, ?)", (expense_id, user_id, amount, 1,exp_date)) # inserting expense

        for debtor in p:
            expense_id = shortuuid.uuid() # expense id of the debtop
            cur.execute("INSERT INTO USER_HAS_GROUP_EXPENSE VALUES (? , ? , ?);", (debtor, group_id, expense_id)) 
            cur.execute("INSERT INTO EXPENSE VALUES (?, ?, ?, ?, ?)", (expense_id, user_id, amount, 0, exp_date)) 

        

In [16]:
conn.commit()

In [13]:
# TRUNCATING THE tables
# cur.execute("TRUNCATE TABLE USER;")
# cur.execute("TRUNCATE TABLE USER_FRIEND;")
# cur.execute("ALTER TABLE TRUNCATE TABLE HAS_GROUP;")


In [14]:
# free resources
# cur.close()
# conn.close()